In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import KFold

In [3]:
data_dir = "../data/raw"
train_adc = pd.read_csv(Path(data_dir) / "train_adc_info.csv")

In [11]:
star_1_ids = train_adc[train_adc.star == 0].planet_id
star_0_ids = train_adc[train_adc.star == 1].planet_id

In [33]:
kfold = KFold(n_splits=5, shuffle=False)

star_0_fold_train = []
star_0_fold_test = []
star_1_fold_train = []
star_1_fold_test = []

for train_index, val_index in kfold.split(star_0_ids):
    star_0_fold_train.append(list(star_0_ids.iloc[train_index]))
    star_0_fold_test.append(list(star_0_ids.iloc[val_index]))

for train_index, val_index in kfold.split(star_1_ids):
    star_1_fold_train.append(list(star_1_ids.iloc[train_index]))
    star_1_fold_test.append(list(star_1_ids.iloc[val_index]))

# Add unseen star in to the validation sets
for fold in star_0_fold_test:
    fold.extend(star_1_ids.sample(n=50))

for fold in star_1_fold_test:
    fold.extend(star_0_ids.sample(n=47))

In [56]:
# Check we did the right thing:

for i in range(5):

    print(
        f"Star 0 folds. Train fold {i} : {train_adc[train_adc.planet_id.isin(star_0_fold_train[i])].star.value_counts()}"
    )
    print(
        f"Star 0 folds. Val fold {i} : {train_adc[train_adc.planet_id.isin(star_0_fold_test[i])].star.value_counts()}"
    )

    print(
        f"Star 1 folds. Train fold {i}: {train_adc[train_adc.planet_id.isin(star_1_fold_train[i])].star.value_counts()}"
    )
    print(
        f"Star 1 folds. Val fold {i}: {train_adc[train_adc.planet_id.isin(star_1_fold_test[i])].star.value_counts()}"
    )

Star 0 folds. Train fold 0 : star
1    261
Name: count, dtype: int64
Star 0 folds. Val fold 0 : star
1    66
0    50
Name: count, dtype: int64
Star 1 folds. Train fold 0: star
0    276
Name: count, dtype: int64
Star 1 folds. Val fold 0: star
0    70
1    47
Name: count, dtype: int64
Star 0 folds. Train fold 1 : star
1    261
Name: count, dtype: int64
Star 0 folds. Val fold 1 : star
1    66
0    50
Name: count, dtype: int64
Star 1 folds. Train fold 1: star
0    277
Name: count, dtype: int64
Star 1 folds. Val fold 1: star
0    69
1    47
Name: count, dtype: int64
Star 0 folds. Train fold 2 : star
1    262
Name: count, dtype: int64
Star 0 folds. Val fold 2 : star
1    65
0    50
Name: count, dtype: int64
Star 1 folds. Train fold 2: star
0    277
Name: count, dtype: int64
Star 1 folds. Val fold 2: star
0    69
1    47
Name: count, dtype: int64
Star 0 folds. Train fold 3 : star
1    262
Name: count, dtype: int64
Star 0 folds. Val fold 3 : star
1    65
0    50
Name: count, dtype: int64
Star 

In [60]:
star_0_fold_train = pd.DataFrame(star_0_fold_train)
star_0_fold_test = pd.DataFrame(star_0_fold_test)
star_1_fold_train = pd.DataFrame(star_1_fold_train)
star_1_fold_test = pd.DataFrame(star_1_fold_test)

In [61]:
star_0_fold_train.to_csv("../config/star_0_train_indices.csv")
star_0_fold_test.to_csv("../config/star_0_test_indices.csv")
star_1_fold_train.to_csv("../config/star_1_train_indices.csv")
star_1_fold_test.to_csv("../config/star_1_test_indices.csv")